# Identificação facial e Reconheicimento facial

In [5]:
import cv2
import face_recognition
import os

def setup_directories(base_path):
    """Cria diretórios necessários para armazenar as imagens das faces."""
    os.makedirs(os.path.join(base_path, 'Face_detectada'), exist_ok=True)
    os.makedirs(os.path.join(base_path, 'Face_documento'), exist_ok=True)

def extract_face_from_document(document_path, save_path):
    """Extrai a face da imagem do documento e salva."""
    document_image = face_recognition.load_image_file(document_path)
    document_face_locations = face_recognition.face_locations(document_image)
    if document_face_locations:
        top, right, bottom, left = document_face_locations[0]
        document_face_image = document_image[top:bottom, left:right]
        document_face_image_bgr = cv2.cvtColor(document_face_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(save_path, document_face_image_bgr)
        print("Face extraída do documento e salva.")
    else:
        print("Nenhum rosto encontrado no documento.")
        return False
    return True

def capture_face_from_webcam(video_capture, save_path):
    """Captura a face da webcam e salva quando o usuário pressiona 'a'."""
    print("Ajuste-se na câmera e pressione 'a' para capturar a face.")
    while True:
        ret, frame = video_capture.read()
        if not ret:
            continue
        rgb_frame = frame[:, :, ::-1]  # Converte de BGR para RGB
        face_locations = face_recognition.face_locations(rgb_frame)

        # Mostra o frame na janela 'Webcam'
        cv2.imshow('Webcam', frame)

        # Aguarda o usuário pressionar uma tecla
        key = cv2.waitKey(1) & 0xFF
        if key == ord('a'):
            if face_locations:
                for (top, right, bottom, left) in face_locations:
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    face_image = frame[top:bottom, left:right]
                    cv2.imwrite(save_path, face_image)
                    print("Face capturada da webcam e salva.")
                    return True
            else:
                print("Nenhum rosto detectado. Tente novamente.")
        elif key == ord('q'):
            break
    return False


def compare_faces(image_path1, image_path2):
    """Compara duas faces e imprime se são da mesma pessoa ou não."""
    image1 = face_recognition.load_image_file(image_path1)
    image2 = face_recognition.load_image_file(image_path2)
    encoding1 = face_recognition.face_encodings(image1)[0]
    encoding2 = face_recognition.face_encodings(image2)[0]
    results = face_recognition.compare_faces([encoding1], encoding2)
    if results[0]:
        print("Os rostos são da mesma pessoa.")
    else:
        print("Os rostos não são da mesma pessoa.")

In [6]:
# Configurações iniciais
base_path = '../src/data'
setup_directories(base_path)
document_path = os.path.join(base_path, 'Documento_original/document.png')
document_face_path = os.path.join(base_path, 'Face_documento/document_face.jpg')
captured_face_path = os.path.join(base_path, 'Face_detectada/captured_face.jpg')

# Processo de extração e captura
if extract_face_from_document(document_path, document_face_path):
    video_capture = cv2.VideoCapture(0)
    if capture_face_from_webcam(video_capture, captured_face_path):
        compare_faces(document_face_path, captured_face_path)
    video_capture.release()
    cv2.destroyAllWindows()

Face extraída do documento e salva.
Ajuste-se na câmera e pressione 'a' para capturar a face.
Face capturada da webcam e salva.
Os rostos são da mesma pessoa.


# Identificação de vivacidade por piscada

In [8]:
import wget
import os

# URL do modelo de marcos faciais
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
model_path = "../src/models/shape_predictor_68_face_landmarks.dat.bz2"  # Caminho para salvar o arquivo baixado

# Baixar o arquivo se não estiver no diretório
if not os.path.exists(model_path):
    wget.download(url, model_path)

# Descompactar o arquivo .bz2
import bz2
uncompressed_path = "../src/models/shape_predictor_68_face_landmarks.dat"  # Nome do arquivo descompactado
if not os.path.exists(uncompressed_path):
    with bz2.BZ2File(model_path, 'rb') as f:
        content = f.read()
        with open(uncompressed_path, 'wb') as new_file:
            new_file.write(content)

# Remover o arquivo .bz2 após descompactação
os.remove(model_path)


In [2]:
import cv2
import face_recognition
import dlib
from scipy.spatial import distance as dist

# Carregar o preditor de marcos faciais
predictor_path = "../src/models/shape_predictor_68_face_landmarks.dat"  # Atualize com o caminho correto
predictor = dlib.shape_predictor(predictor_path)

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

def main():
    video_capture = cv2.VideoCapture(0)
    blinks = 0
    blink_started = False

    print("Por favor, Pisque 3x")

    while blinks < 3:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break

        rgb_frame = frame[:, :, ::-1]  # Conversão de BGR para RGB
        face_locations = face_recognition.face_locations(rgb_frame)
        for face_location in face_locations:
            top, right, bottom, left = face_location
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

            # Identificar e desenhar marcos faciais
            face_landmarks = face_recognition.face_landmarks(rgb_frame, [face_location])
            for face_landmark in face_landmarks:
                left_eye = face_landmark['left_eye']
                right_eye = face_landmark['right_eye']
                left_ear = eye_aspect_ratio(left_eye)
                right_ear = eye_aspect_ratio(right_eye)
                ear = (left_ear + right_ear) / 2.0

                if ear < 0.22:  # Limiar de EAR para detectar piscada
                    if not blink_started:
                        blink_started = True
                elif blink_started:
                    blinks += 1
                    blink_started = False
                    print(f"Piscada {blinks}/3 detectada")

        cv2.imshow('Webcam', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    print("Validação concluída!")
    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Por favor, Pisque 3x
Piscada 1/3 detectada
Piscada 2/3 detectada
Piscada 3/3 detectada
Validação concluída!


: 

In [ ]:
import cv2
import face_recognition
import dlib
from scipy.spatial import distance as dist

# Carregar o preditor de marcos faciais
predictor_path = "../src/models/shape_predictor_68_face_landmarks.dat"  # Substitua pelo caminho correto do seu arquivo
predictor = dlib.shape_predictor(predictor_path)

def calculate_angle(lefteye_center, righteye_center, nose_tip):
    # Calcular o vetor do olho esquerdo para o direito
    eye_vector = righteye_center - lefteye_center
    # Calcular o vetor do centro dos olhos para a ponta do nariz
    nose_vector = nose_tip - ((lefteye_center + righteye_center) / 2)
    
    # Calcular o ângulo entre os dois vetores usando produto escalar
    dot_product = np.dot(eye_vector, nose_vector)
    magnitude = np.linalg.norm(eye_vector) * np.linalg.norm(nose_vector)
    if magnitude == 0:
        return 0
    angle = np.arccos(dot_product / magnitude)
    return np.degrees(angle)

def main():
    video_capture = cv2.VideoCapture(0)
    turned_right = False

    print("Por favor, vire o rosto para a direita")

    while not turned_right:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break

        rgb_frame = frame[:, :, ::-1]  # Conversão de BGR para RGB
        face_locations = face_recognition.face_locations(rgb_frame)
        face_landmarks_list = face_recognition.face_landmarks(rgb_frame, face_locations)

        for face_landmarks in face_landmarks_list:
            lefteye_center = np.array(face_landmarks['left_eye']).mean(axis=0)
            righteye_center = np.array(face_landmarks['right_eye']).mean(axis=0)
            nose_tip = np.array(face_landmarks['nose_tip'][0])

            # Calcular o ângulo
            angle = calculate_angle(lefteye_center, righteye_center, nose_tip)

            # Verificar se o rosto está virado suficientemente para a direita
            if angle > 10:  # Ajuste este valor conforme necessário
                turned_right = True
                print("Rosto virado para a direita detectado!")

        cv2.imshow('Webcam', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    if turned_right:
        print("Validação concluída!")
    else:
        print("Não foi possível detectar a ação.")

    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
